In [5]:
import neuroglancer
import numpy as np
import os
import sys

# add path to ac_ngl to sys.path
sys.path.append('/home/wanqing.yu/AC_Project/ac_visualization/')
from ac_ngl import *

/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


Create a new (initially empty) viewer.  This starts a webserver in a background thread, which serves a copy of the Neuroglancer client, and which also can serve local volume data and handles sending and receiving Neuroglancer state updates.

In [6]:
# viewer = neuroglancer.Viewer()

### open a bigkahuna neuroglancer viewer
ip = 'localhost' 
port = 9999
neuroglancer.set_server_bind_address(bind_address=ip,bind_port=port)
viewer=neuroglancer.Viewer()
viewer

http://localhost:9999/v/cd7e7e65f4831c0a96760c159b898032faf3435d/

Print a link to the viewer (only valid while the notebook kernel is running). Note that while the Viewer is running, anyone with the link can obtain any authentication credentials that the neuroglancer Python module obtains. Therefore, be very careful about sharing the link, and keep in mind that sharing the notebook will likely also share viewer links.

In [3]:
viewer

http://127.0.0.1:55960/v/dfdf52280dba57156edbf7709785b4f8be39d60b/

Add some example layers using the precomputed data source (HHMI Janelia FlyEM FIB-25 dataset).

In [7]:
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://http://bigkahuna.corp.alleninstitute.org/ACdata/Users/wanqing/exaSPIM/precomputed')
  s.layers['skeleton'] = neuroglancer.SegmentationLayer(source='precomputed://http://bigkahuna.corp.alleninstitute.org/ACdata/Users/wanqing/Neuroglancer/ExASPIM/skeletons/')
  s.dimensions = neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[2.961, 2.961, 4])


In [90]:
# 864691135122603047	292685	L2a
# 864691135639556411_267207_L5b
rid = 864691136903144370 #864691135162983725 #864691135639556411 # 864691136134678667 #864691135122603047 #864691135755817170	 # 864691134884807418 #864691135162983725 864691135755817170 	
sid = 236197 #267207 # 271518 #292685 #588983 #518848 #495010 588983
cell_type = 'L6short-a' #'L5ET' #'L5b' # 'L6tall-a' #'L2a' 		
soma_pos = [152681, 226226, 19189]

In [91]:
# make a cetain segment visible
with viewer.txn() as s:
  s.layers['segmentation'].segments = [rid]
  s.voxel_coordinates = soma_pos

In [92]:
# set up path for skeletons

import skeleton_plot as skelplot
import skeleton_plot.skel_io as skel_io

#raw skeleton files
skel_path = 'https://storage.googleapis.com/allen-minnie-phase3/minniephase3-emily-pcg-skeletons/minnie_all/v661/skeletons/'

skel_dir = str(rid)+'_'+str(sid) #'864691134884807418_518848'
skel_filename = str(rid)+'_'+str(sid)+'.swc' #'864691134884807418_518848.swc'

# raw skel
skel = skel_io.read_skeleton(skel_path, skel_filename)
# get the axon segment out
axon_indices = np.where(skel.vertex_properties['compartment'] == 2)[0]
# skel.vertices


In [93]:
import navis
skel_navis = navis.read_swc(skel_path+skel_filename)
skel_navis

,
type,navis.TreeNeuron
name,864691136903144370_236197
n_nodes,7805
n_connectors,None
n_branches,39
n_leafs,51
cable_length,11642.525391
soma,[0]
units,1 dimensionless
created_at,2024-07-19 21:52:03.189264


In [94]:
len(skel_navis.segments[40])

61

In [95]:
skel_seg1 = navis.TreeNeuron(skel_navis.nodes[skel_navis.nodes.node_id.isin(skel_navis.segments[0])])
skel_seg1



/Users/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/navis/graph/graph_utils.py:480: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,
type,navis.TreeNeuron
name,None
n_nodes,708
n_connectors,None
n_branches,0
n_leafs,1
cable_length,1054.874756
soma,[0]
units,1 dimensionless


In [96]:
skel_seg1.nodes

,node_id,label,x,y,z,radius,parent_id,type
0,0,1,609.664001,904.512024,772.400024,5.805,-1,root
1,1,2,609.341980,911.786011,774.395020,0.224,0,slab
2,2,2,609.278015,913.241028,774.794983,0.224,1,slab
3,3,2,609.213013,914.695984,775.193970,0.224,2,slab
4,4,2,609.148987,916.151001,775.593018,0.224,3,slab
...,...,...,...,...,...,...,...,...
703,703,2,1456.977051,774.052979,769.138977,0.139,702,slab
704,704,2,1458.479980,773.953979,769.028992,0.139,703,slab
705,705,2,1459.972046,773.958008,768.812012,0.139,704,slab
706,706,2,1461.462036,774.013977,768.599976,0.139,705,slab


In [97]:
skel_seg1.nodes['x'], skel_seg1.nodes['y'], skel_seg1.nodes['z'] = skel_seg1.nodes['x'] * 1000 / 4, skel_seg1.nodes['y'] * 1000 / 4, skel_seg1.nodes['z'] * 1000 / 40
skel_seg1.nodes

/var/folders/0c/k95xpw3j5k9_9qd8wbmg4z240000gp/T/ipykernel_45087/1175870376.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,node_id,label,x,y,z,radius,parent_id,type
0,0,1,152416.00,226128.00,19310.000000,5.805,-1,root
1,1,2,152335.50,227946.50,19359.875000,0.224,0,slab
2,2,2,152319.50,228310.25,19369.875000,0.224,1,slab
3,3,2,152303.25,228674.00,19379.849609,0.224,2,slab
4,4,2,152287.25,229037.75,19389.824219,0.224,3,slab
...,...,...,...,...,...,...,...,...
703,703,2,364244.25,193513.25,19228.474609,0.139,702,slab
704,704,2,364620.00,193488.50,19225.724609,0.139,703,slab
705,705,2,364993.00,193489.50,19220.300781,0.139,704,slab
706,706,2,365365.50,193503.50,19215.000000,0.139,705,slab


In [98]:
import copy
import time

def interpolate_to(final_state, frames_per_second=5, seconds=1):
    total_frames = int(round(seconds * frames_per_second))
    initial_state = viewer.state
    for frame_i in range(total_frames):
        t = frame_i / total_frames
        viewer.set_state(
            neuroglancer.ViewerState.interpolate(initial_state, final_state, t)
        )
        time.sleep(1 / frames_per_second)
    viewer.set_state(final_state)

def move_by(offset, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates += offset
    interpolate_to(final_state, **kwargs)

def move_to(final_voxel_coordinates, final_orientation, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates = final_voxel_coordinates
    final_state.crossSectionOrientation = final_orientation
    final_state.projectionOrientation = final_orientation
    interpolate_to(final_state, **kwargs)

def do_move_by():
    move_by([100, 100, 100], seconds=1, frames_per_second=10)

def do_move_to(final_voxel_coordinates, final_orientation, seconds=1, frames_per_second=10):
    move_to(final_voxel_coordinates, final_orientation = final_orientation, seconds=seconds, frames_per_second=frames_per_second)


In [99]:
len(skel_seg1.nodes)

708

In [100]:
# move along the axon
# set the initial state
with viewer.txn() as s:
    s.voxel_coordinates = [skel_seg1.nodes['x'][0], skel_seg1.nodes['y'][0], skel_seg1.nodes['z'][0]]

In [66]:
# do move to according to the list of positions and orientations in df
for i in range(40): #len(skel_seg1.nodes)
    do_move_to([skel_seg1.nodes['x'][i], skel_seg1.nodes['y'][i], skel_seg1.nodes['z'][i]], final_orientation = [0,0,1,0], seconds = 1)

KeyboardInterrupt: 

In [101]:
# smooth the skeleton
skel_seg1_sm = navis.smooth_skeleton(skel_seg1, window=10, to_smooth=['x', 'y', 'z'])
fig = skel_seg1_sm.plot3d(backend='plotly', connectors=False)

Smoothing:   0%|          | 0/1 [00:00<?, ?it/s]

In [102]:
# get the dotprops
skel_dp = navis.make_dotprops(skel_seg1, k=10)
skel_dp

,
type,navis.Dotprops
name,None
k,10
units,1 dimensionless
n_points,708


In [103]:
import numpy as np

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

def surface_normal_to_quaternion(normal):
    normal = normalize(normal)
    up_vector = np.array([0, 0, 1])
    axis = np.cross(up_vector, normal)
    axis = normalize(axis)
    angle = np.arccos(np.dot(up_vector, normal))
    
    qx = axis[0] * np.sin(angle / 2)
    qy = axis[1] * np.sin(angle / 2)
    qz = axis[2] * np.sin(angle / 2)
    qw = np.cos(angle / 2)
    
    return [qx, qy, qz, qw]


In [104]:
skel_dp.vect[0]

array([-0.1674677 , -0.9851593 , -0.03762618], dtype=float32)

In [105]:
import math

def angle_between_vectors(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    cos_angle = np.clip(cos_angle, -1, 1)  # Ensure within valid range for arccos
    angle_radians = np.arccos(cos_angle)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

def flip_vectors_if_necessary(skel_dp_vect):
    """Flip the direction of vectors if the angle with the previous vector is > 150 degrees."""
    for i in range(1, len(skel_dp_vect)):
        angle = angle_between_vectors(skel_dp_vect[i], skel_dp_vect[i-1])
        if angle > 150:
            skel_dp_vect[i] = abs(skel_dp_vect[i]) * np.sign(skel_dp_vect[i-1])
    return skel_dp_vect

In [106]:
skel_dp.vect = flip_vectors_if_necessary(skel_dp.vect)

In [107]:
# get the quaternion for each node
quaternions = []
for i in range(len(skel_dp)):
    quaternions.append(surface_normal_to_quaternion(skel_dp.vect[i]))
quaternions

[[0.7101000817391752, -0.12071025712659869, 0.0, 0.6936763710394402],
 [0.7101000817391752, -0.12071025712659869, 0.0, 0.6936763710394402],
 [0.7101000817391752, -0.12071025712659869, 0.0, 0.6936763710394402],
 [0.660935052468352, -0.2943060586765575, 0.0, 0.6903251409625232],
 [0.6609350501164853, -0.29430605762930084, 0.0, 0.6903251436607374],
 [0.6609350432710548, -0.29430607616683063, 0.0, 0.6903251423116303],
 [0.6266516022560623, -0.36399259453034555, 0.0, 0.6890697791348692],
 [0.6266516022560623, -0.36399259453034555, 0.0, 0.6890697791348692],
 [0.6017768265811145, -0.40515871816437415, 0.0, 0.6882667099935638],
 [0.5927382218784017, -0.41774955146959525, 0.0, 0.6885831195806231],
 [0.6045220596688805, -0.3979929436276185, 0.0, 0.6900396337865803],
 [0.6362686580749889, -0.3408528623480465, 0.0, 0.6920849087941425],
 [0.669929374346542, -0.2629322327862471, 0.0, 0.6943063260187751],
 [0.694937064871451, -0.17810468808942295, 0.0, 0.69666433520629],
 [0.7101793483271107, -0.0880

In [108]:
# move along the axon
# set the initial state
with viewer.txn() as s:
    s.voxel_coordinates = [skel_seg1.nodes['x'][0], skel_seg1.nodes['y'][0], skel_seg1.nodes['z'][0]]
    s.crossSectionOrientation = quaternions[0]

In [109]:
# do move to according to the list of positions and orientations in df
for i in range(60): #len(skel_seg1.nodes)
    do_move_to([skel_seg1.nodes['x'][i], skel_seg1.nodes['y'][i], skel_seg1.nodes['z'][i]], final_orientation = quaternions[i], seconds = 2.5)

Create a publicly sharable URL to the viewer state (only works for external data sources, not layers served from Python).  The Python objects for representing the viewer state (`neuroglancer.ViewerState` and friends) can also be used independently from the interactive Python-tied viewer to create Neuroglancer links.

In [64]:
print(neuroglancer.to_url(viewer.state))

https://neuroglancer-demo.appspot.com#!%7B%22dimensions%22:%7B%22x%22:%5B4e-09,%22m%22%5D,%22y%22:%5B4e-09,%22m%22%5D,%22z%22:%5B4e-08,%22m%22%5D%7D,%22position%22:%5B152416,226128,19310%5D,%22crossSectionOrientation%22:%5B0.7101000547409058,-0.12071026861667633,0,0.6936763525009155%5D,%22crossSectionScale%22:1.7160068621848537,%22projectionOrientation%22:%5B0,0,1,0%5D,%22projectionScale%22:524288,%22layers%22:%5B%7B%22type%22:%22image%22,%22source%22:%22precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em%22,%22tab%22:%22source%22,%22name%22:%22image%22%7D,%7B%22type%22:%22segmentation%22,%22source%22:%22precomputed://https://storage.googleapis.com/iarpa_microns/minnie/minnie65/seg_m943/%22,%22tab%22:%22source%22,%22selectedAlpha%22:0.3,%22segments%22:%5B%22864691136903144370%22%5D,%22name%22:%22segmentation%22%7D%5D,%22layout%22:%224panel%22%7D


Stop the Neuroglancer web server, which invalidates any existing links to the Python-tied viewer.

In [85]:
neuroglancer.stop()